In [504]:
pd.set_option('display.max_colwidth', None)

In [1]:
from textblob import TextBlob
import pandas as pd
import sys
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import time
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install gensim
import gensim
from gensim.parsing.preprocessing import remove_stopwords 
import torch
import flair
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/memme11/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Read in sample csv (Twitter analysis)
btc_twitter = pd.read_csv('Bitcoin_tweets.csv')
btc_twitter

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423106,🐥🍀,NaN,#banditgang big win pls 🙏🍀,2018-01-14 07:33:25,17.0,1324.0,1870.0,False,2021-07-02 09:06:24,@earnbnb $bsc $btc #BNB #Bitcoin #BSC #earn...,"['BNB', 'Bitcoin', 'BSC', 'earnBNB', 'fairlaun...",Twitter for iPhone,False
423107,Juliana,NaN,Pejuang rizki allah 🙏\nSemangat ibadah,2021-06-28 05:27:54,11.0,212.0,1058.0,False,2021-07-02 09:06:21,@GiveAwayHost Bismillah\nWish me luck \n#Bitcoin,['Bitcoin'],Twitter for Android,False
423108,Der Zeitgeist,"Kiel, Schleswig-Holstein","Was ihr den Geist der Zeiten heißt, Das ist im...",2013-02-11 15:28:36,1929.0,1803.0,5252.0,False,2021-07-02 09:06:21,What do we learn from this? Don't mess with th...,"['ElSalvador', 'BTC', 'Bitcoin', 'mining', 'IM...",Twitter Web App,False
423109,icon0.com,NaN,"#icon0 Free images for personal , commercial a...",2010-04-11 06:22:55,599.0,15.0,3324.0,False,2021-07-02 09:06:20,Download Bitcoin icon sign design\n Free image...,"['bitcoin', 'btc', 'cryptocurrency', 'crypto',...",Twitter Web App,False


In [3]:
# Clean 
# Drop columns
btc_twitter = btc_twitter.drop(columns=['user_name', 'user_location', 'user_description', 'user_created', 'user_verified', 'source'])

# Clean date and text columns using regex
btc_twitter['date'] = btc_twitter['date'].str.extract('(2021-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')

btc_twitter['cleaned_text'] = btc_twitter['text']
clean_text = '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)'
btc_twitter['cleaned_text'] = btc_twitter['cleaned_text'].str.replace(clean_text, " ", regex=True)
btc_twitter['cleaned_text'] = btc_twitter['cleaned_text'].str.lower()

# Re-order columns 
columns = ['text', 'cleaned_text', 'user_followers', 'user_friends', 'user_favourites', 'hashtags', 'is_retweet', 'date']
btc_twitter = btc_twitter[columns]

# Convert date dtype to datetime
btc_twitter['date'] = pd.to_datetime(btc_twitter['date'])

# Keep only tweets with user_followers > 5000
btc_twitter = btc_twitter.loc[btc_twitter['user_followers']>5000]

btc_twitter

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet,date
0,Blue Ridge Bank shares halted by NYSE after #b...,blue ridge bank shares halted by nyse after b...,8534.0,7605,4838,['bitcoin'],False,2021-02-10 23:59:04
1,"😎 Today, that's this #Thursday, we will do a ""...",today that s this thursday we will do a ...,6769.0,1532,25483,"['Thursday', 'Btc', 'wallet', 'security']",False,2021-02-10 23:58:48
9,.@Tesla’s #bitcoin investment is revolutionary...,s bitcoin investment is revolutionary for ...,8534.0,7605,4838,"['bitcoin', 'crypto']",False,2021-02-10 23:52:08
12,Join our first virtual crypto meetup of 2021 -...,join our first virtual crypto meetup of 2021 ...,5097.0,791,52,NaN,False,2021-02-10 23:50:59
16,@naval #BTC is unconfiscatable \n\nAll roads l...,btc is unconfiscatable all roads lead to ...,72542.0,519,19763,"['BTC', 'Bitcoin']",False,2021-02-10 23:48:37
...,...,...,...,...,...,...,...,...
423045,What Problems Does Bitcoin Solve? https://t.co...,what problems does bitcoin solve cryptocur...,14982.0,5832.0,16169.0,"['cryptocurrency', 'crypto', 'digital', 'bitco...",False,2021-07-02 09:10:02
423046,"Live #Bitcoin Price: $33,377\n\nThat's a decre...",live bitcoin price 33 377 that s a decreas...,39946.0,7518.0,970.0,"['Bitcoin', 'Bitcoin', 'Binance', 'Crypto', 'D...",False,2021-07-02 09:10:02
423065,Bitcoin bounces at key range amid hope BTC pri...,bitcoin bounces at key range amid hope btc pri...,9276.0,175.0,0.0,"['news', 'investing', 'bitcoin']",False,2021-07-02 09:09:15
423092,I heard that the bull market is over.\n\nThe d...,i heard that the bull market is over the dar...,8702.0,159.0,64.0,['Bitcoin'],False,2021-07-02 09:07:04


In [4]:
btc_twitter.dtypes

text                       object
cleaned_text               object
user_followers            float64
user_friends               object
user_favourites            object
hashtags                   object
is_retweet                 object
date               datetime64[ns]
dtype: object

In [5]:
sorted_btc = btc_twitter.set_index('date').sort_index(ascending=False)
sorted_btc.dropna(inplace=True)
sorted_btc.drop_duplicates(inplace=True)
sorted_btc

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet
date,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroa...,thailand boutiquehotels for sale willroad...,9150.0,128.0,47.0,"['Thailand', 'Boutiquehotels', 'willroadhouse'...",False
2021-07-05 23:26:36,LUNALAND ($LLN) will burn coin supply until we...,lunaland lln will burn coin supply until we...,10155.0,7181.0,8833.0,"['presale', 'blockchain', 'cryptotrading', 'in...",False
2021-07-05 23:25:27,#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD ...,lightningnetwork observed 22 365 nodes ...,9975.0,6.0,6.0,"['LightningNetwork', 'Lightning', 'LN', 'bitco...",False
2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is...",barclay s just wants to keep you safe this is...,20668.0,451.0,0.0,['bitcoin'],False
2021-07-05 23:24:35,Why 'Cardano is just teasing us' - https://t.c...,why cardano is just teasing us ada alt...,9276.0,175.0,0.0,"['altcoin', 'news', 'bitcoin']",False
...,...,...,...,...,...,...,...
2021-02-05 11:00:24,"5⃣ #Blockchain 50 2021 by @DelRayMan, @Forbe...",5 blockchain 50 2021 by cry...,16813.0,818,1201,"['Blockchain', 'cryptocurrency', 'bitcoin', 'c...",False
2021-02-05 11:00:24,4⃣ 🎙️ Bloomberg LP #CryptoOutlook 2021 with @...,4 bloomberg lp cryptooutlook 2021 with ...,16813.0,818,1201,"['CryptoOutlook', 'cryptocurrency', 'bitcoin',...",False
2021-02-05 11:00:22,2⃣ Debunking 9 #Bitcoin Myths by @Patrick_Lo...,2 debunking 9 bitcoin myths by lowry ...,16813.0,818,1201,"['Bitcoin', 'cryptocurrency', 'bitcoin', 'cryp...",False


In [6]:
# Output to csv
output_file = 'Cleaned_Bitcoin_tweets.csv'
sorted_btc.to_csv(output_file, index='inplace')

# Sentiment Analysis #1 - Using vader + nltk

In [524]:
# Sentiment
tweets = sorted_btc.head(500)

In [525]:
#Functions for tokenization, stopwords and stemmer
def tokenization(text):
    text = re.split('\W+', text)
    return text


#Removing stopwords
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

#Applying Stemmer
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

In [526]:
# Execute functions
tweets['tokenized'] = tweets['cleaned_text'].apply(lambda x: tokenization(x.lower()))
tweets['nonstop'] = tweets['tokenized'].apply(lambda x: remove_stopwords(x))
tweets['stemmed'] = tweets['nonstop'].apply(lambda x: stemming(x))

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

In [527]:
tweets

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet,tokenized,nonstop,stemmed
date,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",False,"[, thailand, boutiquehotels, for, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency, ]","[, thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency, ]","[, thailand, boutiquehotel, sale, willroadhous, aoyroadhous, compassgroupintern, thailandbeachfronthom, thailandbeachfronthom, retir, thailandrealest, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpat, bitcoin, cryptocurr, ]"
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",False,"[lunaland, lln, will, burn, coin, supply, until, we, reach, 1, lunar, distance, in, coins, 400m, coins, a, burn, rate, of, 99, 96, from, the, initial, pool, of, 1, trillion, coins, reduces, coin, supply, presale, blockchain, cryptotrading, investing, cryptocurrency, bitcoin, trading, altcoin, cryptonews]","[lunaland, lln, burn, coin, supply, reach, 1, lunar, distance, coins, 400m, coins, burn, rate, 99, 96, initial, pool, 1, trillion, coins, reduces, coin, supply, presale, blockchain, cryptotrading, investing, cryptocurrency, bitcoin, trading, altcoin, cryptonews]","[lunaland, lln, burn, coin, suppli, reach, 1, lunar, distanc, coin, 400m, coin, burn, rate, 99, 96, initi, pool, 1, trillion, coin, reduc, coin, suppli, presal, blockchain, cryptotrad, invest, cryptocurr, bitcoin, trade, altcoin, cryptonew]"
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,9975.0,6.0,6.0,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']",False

In [528]:
# Sentiment

tweets[['polarity', 'subjectivity']] = tweets['cleaned_text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tweets['cleaned_text'].iteritems():
    
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    
    if pos > neg:
        tweets.loc[index, 'sentiment'] = 'positive'
        
    elif neg > pos:
        tweets.loc[index, 'sentiment'] = 'negative'
    
    else:
        tweets.loc[index, 'sentiment'] = 'neutral'
        
    tweets.loc[index, 'negative'] = neg
    tweets.loc[index, 'neutral'] = neu
    tweets.loc[index, 'positive'] = pos
    tweets.loc[index, 'compound'] = comp


/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

...

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is tr

In [529]:
tweets.head(30)

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet,tokenized,nonstop,stemmed,polarity,subjectivity,sentiment,negative,neutral,positive,compound
date,,,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",False,"[, thailand, boutiquehotels, for, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency, ]","[, thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency, ]","[, thailand, boutiquehotel, sale, willroadhous, aoyroadhous, compassgroupintern, thailandbeachfronthom, thailandbeachfronthom, retir, thailandrealest, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpat, bitcoin, cryptocurr, ]",0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",False,"[lunaland, lln, will, burn, coin, supply, until, we, reach, 1, lunar, distance, in, coins, 400m, coins, a, burn, rate, of, 99, 96, from, the, initial, pool, of, 1, trillion, coins, reduces, coin, supply, presale, blockchain, cryptotrading, investing, cryptocurrency, bitcoin, trading, altcoin, cryptonews]","[lunaland, lln, burn, coin, supply, reach, 1, lunar, distance, coins, 400m, coins, burn, rate, 99, 96, initial, pool, 1, trillion, coins, reduces, coin, supply, presale, blockchain, cryptotrading, investing, cryptocurrency, bitcoin, trading, altcoin, cryptonews]","[lunaland, lln, burn, coin, suppli, reach, 1, lunar, distanc, coin, 400m, coin, burn, rate, 99, 96, initi, pool, 1, trillion, coin, reduc, coin, suppli, presal, blockchain, cryptotrad, invest, cryptocurr, bitcoin, trade, altcoin, cryptonew]",0.000000,0.000000,positive,0.000,0.974,0.026,0.0258
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median n

In [530]:
tweets['sentiment'].value_counts()

positive    291
negative    105
neutral     104
Name: sentiment, dtype: int64

# Sentiment Analysis Trial #2 - Flair built-in model

In [531]:
tweets2 = sorted_btc.head(500)
tweets2

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet
date,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",False
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",False
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,9975.0,6.0,6.0,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']",False
2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs",barclay s just wants to keep you safe this is why they let you spend your life savings on the lottery at the casino or trading stock futures but buying bitcoin on binance is a bridge to far ps they are a huge criminal money launderer,20668.0,451.0,0.0,['bitcoin'],False
2021-07-05 23:24:35,Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin,why cardano is just teasing us ada altcoin news bitcoin,9276.0,175.0,0.0,"['altcoin', 'news', 'bitcoin']",False
...,...,...,...,...,...,...,...
2021-07-05 20:24:17,#linkedin #twitter #giftidea #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e,linkedin twitter giftidea facebook instagram tumblr socialmedia pinterest dogecoin reddit gift gifts giftideas shop shopping affiliatemarketing bitcoin shop gifts amp baskets,67458.0,6258.0,23.0,"['linkedin', 'twitter', 'giftidea', 'facebook', 'instagram', 'tumblr', 'socialmedia', 'pinterest', 'dogecoin', 'reddit', 'gift', 'gifts', 'giftideas', 'shop', 'shopping', 'affiliatemarketing', 'bitcoin', 'gifts', 'baskets']",False
2021-07-05 20:23:10,"Argentina vs. Colombia: Copa America semifinals live stream, TV channel, how to watch online, news, odds https://t.co/ajfyDCgPlj ▶️ https://t.co/NKkDBVS66N #

In [532]:
# Load built-in flair sentiment model
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-07-16 01:18:46,701 loading file /Users/memme11/.flair/models/sentiment-en-mix-distillbert_4.pt


In [533]:
# Sentiment #2 - using built in flair-sentiment model
probs = []
sentiments = []

for tweet in tweets2['cleaned_text'].to_list():
    
    # make prediction
    sentence = flair.data.Sentence(tweet)
    sentiment_model.predict(sentence)
    
    # extract sentiment prediction
    probs.append(sentence.labels[0].score)  # numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

# add probability and sentiment predictions to tweets dataframe
tweets2['probability'] = probs
tweets2['sentiment'] = sentiments

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [534]:
tweets2.head(30)

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet,probability,sentiment
date,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",False,0.998792,POSITIVE
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",False,0.999127,NEGATIVE
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,9975.0,6.0,6.0,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']",False,0.528815,POSITIVE
2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs",barclay s just wants to keep you safe this is why they let you spend your life savings on the lottery at the casino or trading stock futures but buying bitcoin on binance is a bridge to far ps they are a huge criminal money launderer,20668.0,451.0,0.0,['bitcoin'],False,0.997891,NEGATIVE
2021-07-05 23:24:35,Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin,why cardano is just teasing us ada altcoin news bitcoin,9276.0,175.0,0.0,"['altcoin', 'news', 'bitcoin']",False,0.989429,NEGATIVE
2021-07-05 23:24:26,Ethereum Exchange Reserves Plummet To More Than Two-Year Low #Ethereum #bitcoin via https://t.co/gwG7Duur7Z https://t.co/eDz6P1tFqb,ethereum exchange reserves plummet to more than two year low ethereum bitcoin via,14169.0,7424.0,12101.0,"['Ethereum', 'bitcoin']",False,0.999962,NEGATIVE
2021-07-05 23:24:10,Who’s ready to win $100 of #Bitcoin right now 😎🤑,who s ready to win 100 of bitcoin right now,8135.0,6.0,1030.0,['Bitcoin'],False,0.992545,POSITIVE
2021-07-05 23:24:10,#linkedin #twitter #giftsforhim #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e,linkedin twitter giftsforhim facebook instagra

In [535]:
tweets2['sentiment'].value_counts()

POSITIVE    267
NEGATIVE    233
Name: sentiment, dtype: int64